# Funciones para Detección de Outliers

## Se crean dos funciones para identificar Outliers y una funcion para visualizar graficamente la serie de tiempo y sus valores atipicos:

1. Detección de Outliers con método Tukey.
2. Función que itera sobre todos los SKU y crea una nueva columna con los outliers detectados por día.

No se eliminan ni se reemplazan los Outliers, ya que eso depende del conocimiento de cada negocio. El objetivo es identificarlos estadísticamente y que sea el usuario los que decidan editarlos en una interfaz gráfica. El resultado final será la demanda ajustada a los que se le aplicará los distintos métodos de proyección de demanda.

## FUNCION 1 PARA DETECTAR OUTLIERS POR METODO TUKEY

In [ ]:
import numpy as np

def outliers_tukey(serie):
    '''Detecta los "outliers" en la Serie de Tiempo usando
    el método de Tukey.
    
    Entradas:
    - serie: la Serie de Tiempo (DataFrame de Pandas)
    
    Salida:
    - outliers: DataFrame de Pandas con los valores extremos detectados
    '''
    
    # 1. Calcular Q1 y Q3
    q1, q3 = np.percentile(serie, [25, 75])
    
    # 2. Calcular IQR
    IQR = q3 - q1
    
    # 3. Calcular límites superior e inferior
    lim_sup = q3 + 1.5*IQR
    lim_inf = q1 - 1.5*IQR
    
    # 4. Filtrar serie de tiempo para detectar los outliers
    outliers = serie[(serie.to_numpy() > lim_sup) | (serie.to_numpy() < lim_inf)]
    
    return outliers

Ejemplo de Como usar la función con un solo SKU.

Esta función toma el return de la función 'preparar_series_tiempo'

In [ ]:
## Ejemplo de Como usar la funcion con un solo SKU
outliers = outliers_tukey(df_filtered['Quantity'])

## FUNCION 2 PARA DETECTAR OUTLIERS POR CADA SKU

In [2]:
import numpy as np
import pandas as pd

def detectar_outliers_tukey(df, col_sku='ItemCode', col_qty='Quantity', col_outlier='Outlier'):
    """
    Detecta outliers en un DataFrame de series de tiempo por SKU usando el método de Tukey.
    
    Parámetros:
    - df: DataFrame con DatetimeIndex, columna SKU y columna de valores
    - col_sku: nombre de la columna con los SKU
    - col_qty: nombre de la columna con los valores
    - col_outlier: nombre de la columna que contendrá los valores detectados como outliers
    
    Devuelve:
    - df con una columna extra col_outlier: valores si son outliers, NaN si no
    """
    
    # Inicializamos columna de outliers con NaN
    df[col_outlier] = np.nan
    
    # Función interna para detectar outliers por serie
    def outliers_tukey(serie):
        q1, q3 = np.percentile(serie, [25, 75])
        IQR = q3 - q1
        lim_inf = q1 - 1.5*IQR
        lim_sup = q3 + 1.5*IQR
        out = serie[(serie < lim_inf) | (serie > lim_sup)]
        return out
    
    # Aplicamos por SKU
    for sku, grupo in df.groupby(col_sku):
        out = outliers_tukey(grupo[col_qty])
        df.loc[out.index, col_outlier] = out
    
    return df

Ejemplo de Como usar la función para todos los SKU resultado de la función 'preparar_series_tiempo'

In [ ]:
# df_series es el DataFrame que devolvió preparar_series_tiempo
df_con_outliers = detectar_outliers_tukey(df)

## FUNCION 3 GRAFICA DE SERIE DE TIEMPO CON OUTLIERS

En caso de querer filtrar por un solo producto y visualizar la grafica de la serie de tiempo y sus valores Outliers.

In [ ]:
# Filtro de producto
df_filtered = df_con_outliers[df_con_outliers['ItemCode'].isin(['1001005059'])]
#df_filtered['ItemCode'].value_counts()

In [ ]:
# Función para Graficas Outlier
import matplotlib.pyplot as plt

def graficar_outliers(serie, outliers):
    """
    Graficar la Serie de Tiempo y superponer los outliers detectados.
    
    Parámetros:
    - serie: Serie de tiempo (pandas Series)
    - outliers: Serie de outliers detectados (pandas Series)
    """
    
    plt.figure(figsize=(14,6))
    
    # Graficar la serie de tiempo
    plt.plot(serie.index, serie.values, label='Serie de Tiempo', color='blue', alpha=0.7)
    
    # Graficar los outliers detectados
    plt.scatter(outliers.index, outliers.values, color='red', label='Outliers', s=50)
    
    # Ajustar etiquetas y leyenda
    plt.xlabel('Fecha')
    plt.ylabel('Valores')
    plt.title('Serie de Tiempo con Outliers Detectados', fontsize=14)
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
serie = df_filtered['Quantity']
outliers = df_filtered['Outlier'].dropna()

graficar_outliers(serie, outliers)